In [2]:
# Importamos nuestra librerías básicas
import numpy as np
import os

# Este paso es para que los resultados en vuestros notebooks sean iguales a lo de este
np.random.seed(42)


# Aprendizaje supervisado - SVM

En este notebook volveremos a clasificar el MNIST, pero ahora utilizando el clasificador basado en Support Vector Machine o SVM

In [3]:
# Aqui importamos los datos que vamos a clasificar

# Scikit-Learn ya incluye algunos datasets de ejemplo como el MNIST
from sklearn.datasets import fetch_mldata
mnist = fetch_mldata('MNIST original')

# El dataset contiene 70.000 ejemplos de digitos escritos a mano
# en blanco y negro. Cada digito esta representada por una imagen de 28x28 pixels.
# Además el dataset incluye el "target" i.e. el numero asociado con cada imagen.

# Otros datasets serían más rápidos de ejecutar, pero la ventaja de este es que,
# debido a su complejidad, cuando empecemos a utilizar modelos más complejos (como 
# redes neuronales), se empezarán a ver las ventajas de estos modelos más complejos

# Aqui cargamos nuestros ejemplos en X, el target en y. Nuestro objetivo con
# Machine Learning es aprender la función f(X) que genera y.
X, y = mnist["data"], mnist["target"]

In [4]:
# Nuestro objetivo es que la función aprendida funcione no solamente con
# el dataset de prueba, pero que también "generalize" bien para ejemplos
# que no haya visto antes.

# En esta sección separamos el dataset en 2 partes: el training set y test set
X_train_o, X_test_o, y_train, y_test = X[:60000], X[60000:], y[:60000], y[60000:]

### Normalización de los datos

In [5]:
# Antes de enviar los datos al modelo, hacemos una adaptación de los datos para normalizarlos
# Por normalización en este contexto la idea es remover la media de cada "feature"
# (i.e. centrar en zero) y dividir por la "variance", es decir hacer que los datos estén entre
# 0 y 1.

from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train_o)
X_test = scaler.transform(X_test_o)


C:\ProgramData\Anaconda3\envs\tensorflow\lib\site-packages\sklearn\utils\validation.py:475: DataConversionWarning: Data with input dtype uint8 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)


## Clasificador utilizando Support Vector Machines

In [7]:
# El modelo SVC ...

from sklearn.svm import SVC

# Creamos el modelo
svc_clf = SVC(probability=True, random_state=42)

# Hacemos fit del modelo al dato
%time svc_clf.fit(X_train, y_train)

# Medimos el rendimiento.
%time train_score = svc_clf.score(X_train, y_train)
print("Rendimiento en el dataset de training: %.4f" % train_score)
%time score = svc_clf.score(X_test, y_test)
print("Rendimiento en el dataset de pruebas: %.4f" % score)

Wall time: 1h 32s
Wall time: 15min 38s
Rendimiento en el dataset de training: 0.9853
Wall time: 2min 35s
Rendimiento en el dataset de pruebas: 0.9665
